In [1]:
using JuMP, Gurobi, Ipopt, Clp, GLPK, ECOS
using LightGraphs, GraphPlot
using DataFrames
using Statistics

┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1278


# 1. Spatial 7-bus Case

In [1035]:
# scenario_num takes integer value from 1 to 7. Each scenario corresponds to different virtual shift capacity
scenario_num = 6;

In [1036]:
N = 7;
D = [13., 0., 17., 0., 0., 17., 13.];
S = [5., 20., 5., 20., 0., 5., 5.];
#S = [0., 20., 0., 20., 0., 0., 0.];
α_d = [10., 0., 10., 0., 0., 15., 15.];
α_s = [3., 1., 3., 1., 0., 3., 3.];
dmax = [20., 0., 20., 0., 0., 20., 20.];
# dmax = [25., 0., 25., 0., 0., 25., 25.];

edges = [(1,2),(1,3),(2,3),(3,4),(4,5),(5,6),(5,7),(6,7)];
Dtheta = zeros(Float64, N, N);
B = zeros(Float64, N, N);
α_f = zeros(Float64, N, N);
for (i,j) in edges
    Dtheta[i,j] = 10.;
    B[i,j] = 1.;
    Dtheta[j,i] = 10.;
    B[j,i] = 1.;
    α_f[i,j] = 0.1;
    α_f[j,i] = 0.1;
end

c1_pos = [(1,2), (2,3), (3,1)]
c1_neg = [(2,1), (3,2), (1,3)]
c2_pos = [(5,6), (6,7), (7,5)]
c2_neg = [(7,6), (6,5), (5,7)]

Del = zeros(N,N)          # capacity matrix of virtual links
                          # scenario 1: no spatial shifts
if scenario_num == 2      # scenario 2: δ_{13} = 1
    Del[1,7] = 5;
    Del[7,1] = 5;
#     Del[1,7] = 1000;
#     Del[7,1] = 1000;
elseif scenario_num == 3
    Del[1,7] = 10;
    Del[7,1] = 10;
elseif scenario_num == 4
    Del[1,7] = 5;
    Del[7,1] = 5;
    Del[1,3] = 5;
    Del[3,1] = 5;    
elseif scenario_num >= 5
    Del[1,7] = 10;
    Del[7,1] = 10;
    Del[1,3] = 10;
    Del[3,1] = 10;    
end

α_del = zeros(N,N);         # virtual shift bidding costs
if scenario_num <= 6
    α_del = 0.3 * (copy(Del).>0);
end
if scenario_num >= 6
    dmax = [25., 0., 25., 0., 0., 25., 25.];
end

7-element Array{Float64,1}:
 25.0
  0.0
 25.0
  0.0
  0.0
 25.0
 25.0

In [1037]:
m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag" => 0))
#m = Model(optimizer_with_attributes(Ipopt.Optimizer))
@variable(m, d[1:N] >= 0)
@variable(m, s[1:N] >= 0)
@variable(m, θ[1:N])
@variable(m, f[1:N, 1:N] >= 0)
@variable(m, δ[1:N,1:N] >= 0)
# @variable(m, dθ[1:N, 1:N] >= 0)

@constraint(m, d .<= D)
@constraint(m, s .<= S)
caps = @constraint(m, f .<= B .* Dtheta)
@constraint(m, δ .<= Del)

for i in 1:N, j in (i+1):N
    if B[i,j] > 0
        @constraint(m, f[i,j] - f[j,i] == B[i,j] * (θ[i] - θ[j]))
    end
end

# physics = Dict()
# physics[1] = @constraint(m, dθ[1,2] - dθ[2,1] + dθ[2,3] - dθ[3,2] + dθ[3,1] - dθ[1,3] == 0)
# physics[2] = @constraint(m, dθ[5,6] - dθ[6,5] + dθ[6,7] - dθ[7,6] + dθ[7,5] - dθ[5,7] == 0)

bals = Dict()

for i in 1:N
#     bals[i] = @constraint(m, sum(B[:, i] .* dθ[:, i]) + s[i] + sum(δ[i,:]) ==
#                              sum(B[i, :] .* dθ[i, :]) + d[i] + sum(δ[:,i]))
    bals[i] = @constraint(m, sum(f[:,i]) + s[i] + sum(δ[i,:]) == sum(f[i,:]) + d[i] + sum(δ[:,i]))
end

comp_lb = Dict()
comp_ub = Dict()
for i in 1:N
    comp_lb[i] = @constraint(m, d[i] + sum(δ[:,i]) - sum(δ[i,:]) >= 0)
    comp_ub[i] = @constraint(m, -d[i] - sum(δ[:,i]) + sum(δ[i,:]) >= -dmax[i])
end

@objective(m, Min, sum(α_s .* s) + sum(α_f .* f) - sum(α_d .* d) + sum(α_del .* δ))
optimize!(m)

Academic license - for non-commercial use only - expires 2021-06-10


In [1038]:
flows = B .* value.(f)
prices = [dual(bals[i]) for i in 1:N];
# physics_dual = [dual(physics[i]) for i in 1:2]
f_marginal_prices = zeros(size(flows))
unit_profit = zeros(size(flows))
d_ex = [value(d[i] + sum(δ[:,i]) - sum(δ[i,:])) for i in 1:N];
ω_lb = [dual(comp_lb[i]) for i in 1:N]
ω_ub = [dual(comp_ub[i]) for i in 1:N]
ω = ω_ub .- ω_lb

#=
for i in 1:N, j in 1:N
    if j != i
        unit_profit[i,j] = (prices[j] - prices[i] - α_f[i,j]) * B[i,j]
        f_marginal_prices[i,j] = unit_profit[i,j] + (physics_dual[1] * ((i,j) in c1_pos) -
                                                     physics_dual[1] * ((i,j) in c1_neg) +
                                                     physics_dual[2] * ((i,j) in c2_pos) -
                                                     physics_dual[2] * ((i,j) in c2_neg))
    end
end
=#
#println("Flows are: ", f)
#println("Marginal prices of flows: ", f_marginal_prices)
println("Social welfare: ", -objective_value(m))
println("Prices: ", prices)
# println("Cycle constraint dual: ", physics_dual)
#println("Flow profits: ", f .* f_marginal_prices)
println("Flow values: ", flows)
println("Load Execution: ", d_ex)
println("Virtual links:", value.(δ))
println("Demand: ", value.(d))
println("Supply: ", value.(s))
println("Omega: ", ω_ub .- ω_lb)

Social welfare: 639.1333333333333
Prices: [3.3, 2.7, 3.0, 2.9, 3.533333333333333, 3.6666666666666665, 3.5999999999999996]
Flow values: [0.0 0.0 0.0 0.0 0.0 0.0 0.0; 10.0 0.0 10.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 10.0 0.0 10.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 7.333333333333334 2.666666666666666; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 4.666666666666667 0.0]
Load Execution: [14.999999999999998, 0.0, 25.0, 0.0, 0.0, 17.0, 2.9999999999999982]
Virtual links:[0.0 0.0 8.000000000000002 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 10.000000000000002 0.0 0.0 0.0 0.0 0.0 0.0]
Demand: [13.0, 0.0, 17.0, 0.0, 0.0, 17.0, 13.0]
Supply: [5.0, 20.0, 5.000000000000002, 20.0, 0.0, 5.0, 5.0]
Omega: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [1039]:
load_payment = sum(prices .* value.(d))
transmission_revenue = value(sum((B[i,j] > 0) * f[i,j] * (prices[j] - prices[i]) for i in 1:N for j in 1:N))
gen_revenue = sum(prices .* value.(s))
vl_revenue = value(sum((Del[i,j] > 0) * δ[i,j] * (prices[i] - prices[j]) for i in 1:N for j in 1:N))
total_load = value(sum(d))
println("total load cleared: $(total_load)")
println("total load payment: $(load_payment)")
println("total transmission revenue: $(transmission_revenue)")
println("total generation revenue: $(gen_revenue)")
println("total VL revenue: $(vl_revenue)")
println("total revenue: $(transmission_revenue + gen_revenue + vl_revenue)")
println("average price: $((load_payment - vl_revenue)/total_load)")

total load cleared: 60.0
total load payment: 203.03333333333336
total transmission revenue: 17.799999999999997
total generation revenue: 179.83333333333334
total VL revenue: 5.399999999999998
total revenue: 203.03333333333333
average price: 3.293888888888889


In [989]:
load_profit = sum((α_d .- prices .- ω) .* value.(d))
# vl_profit = value(sum((Del[i,j] > 0) * δ[i,j] * (prices[i] + ω[i] - prices[j] - ω[j] - α_del[i,j]) for i in 1:N for j in 1:N))
vl_profit = value(sum((Del[i,j] > 0) * δ[i,j] * (prices[i] + ω[i] - prices[j] - ω[j] - α_del[i,j]) for i in 1:N for j in 1:N))
grid_profit = value(sum((B[i,j] > 0) * f[i,j] * (prices[j] - prices[i] - α_f[i,j]) for i in 1:N for j in 1:N))
g2_profit = (prices[2] - α_s[2]) * value(s[2])
g4_profit = (prices[4] - α_s[4]) * value(s[4])
println("total load profit: $(total_load)")
println("total vl profit: $(vl_profit)")
println("total grid profit: $(grid_profit)")
println("G2 profit: $(g2_profit)")
println("G4 profit: $(g4_profit)")

total load profit: 60.0
total vl profit: 0.0
total grid profit: 75.33333333333333
G2 profit: 0.0
G4 profit: 0.0


# 2. Temporal Case

In [900]:
scenario_num = 7;

In [901]:
# Node parameters
T = 4                       # Number of times
α_p = [10,20,10,15]      # bidding prices of supply
α_d = [30,60,40,50]      # bidding prices of demand
S = [50,50,50,50]        # supply capacity
D = [70,25,70,40]        # demand capacity
del_cap = Dict((i,j) => 0 for i in 1:T for j in (i+1):T)
# r = [10,10,10,10]
r = [20,20,20,20]        # reserved computing power
Dmax = 70

# Virtual link parameters
if scenario_num == 2
    del_cap[(1,2)] = 8
elseif scenario_num == 3
    del_cap[(1,2)] = 10;
elseif scenario_num == 4
    del_cap[(1,2)] = 21;
elseif scenario_num == 5
    del_cap[(1,2)] = 21;
    del_cap[(1,3)] = 20;
elseif scenario_num == 6
    del_cap[(1,2)] = 11;
    del_cap[(1,4)] = 11;
elseif scenario_num == 7
    del_cap[(1,2)] = 11;
    del_cap[(1,4)] = 11;
    del_cap[(3,4)] = 10;
elseif scenario_num == 8
    del_cap[(1,2)] = 11;
    del_cap[(1,4)] = 11;
    del_cap[(3,4)] = 20;
elseif scenario_num >= 9
    del_cap[(1,2)] = 21;
    del_cap[(1,4)] = 11;
    del_cap[(3,4)] = 20;
end

α_del = Dict((i,j) => 3 for i in 1:T for j in (i+1):T)

# Ramping constraint parameters
R = 15;

In [902]:
# Optimizaiton formulation
vls = keys(del_cap)

m = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0))
@variable(m, p[1:T] >= 0)                         # supply
@variable(m, d[1:T] >= 0)                         # demand
@variable(m, δ[vls] >= 0)                         # virtual shift

# Flow and virtual link constraints
for v in vls
    @constraint(m, δ[v] <= del_cap[v])            # virtual shift capacity constraints
end

# Supply and demand capacity constraints
@constraint(m, p .<= S)
@constraint(m, d .<= D)
for i in 1:T-1
    @constraint(m, d[i] + sum(δ[(k,l)] for (k,l) in vls if l == i) - sum(δ[(k,l)] for (k,l) in vls if k == i) >= 0)
#     @constraint(m, d[i] + sum(δ[(k,l)] for (k,l) in vls if l == i) - sum(δ[(k,l)] for (k,l) in vls if k == i) <= D[i] + r[i])
    @constraint(m, d[i] + sum(δ[(k,l)] for (k,l) in vls if l == i) - sum(δ[(k,l)] for (k,l) in vls if k == i) <= Dmax)
end

# Ramping constraints
for i in 1:T-1
    @constraint(m, p[i+1] - p[i] <= R)
    @constraint(m, p[i+1] - p[i] >= -R)
end

# Node balance constraint
bals = Dict()
for i in 1:T
    bals[i] = @constraint(m, p[i] - d[i] - sum(δ[(k,l)] for (k,l) in vls if l == i) + sum(δ[(k,l)] for (k,l) in vls if k == i) == 0)
end

# Objecitve: social welfare
@objective(m, Min, sum(α_p.*p - α_d.*d) + sum(α_del[v] * δ[v] for v in vls));


Academic license - for non-commercial use only - expires 2021-06-10


In [903]:
optimize!(m)
prices = zeros(T)
for i in 1:T
    prices[i] = dual(bals[i])
end

# Print info
println("The optimal value of virtual link model is ", -objective_value(m))
println("The nodal prices of virtual link model is ", prices)
println("Amt of cleared loads are ", value.(d))
println("The supplies are ", value.(p))
println("The virtual flows are ", value.(δ))

The optimal value of virtual link model is 5197.0
The nodal prices of virtual link model is [30.0, 20.0, 40.0, 27.0]
Amt of cleared loads are [61.0, 25.0, 60.0, 40.0]
The supplies are [50.0, 36.0, 50.0, 50.0]
The virtual flows are 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [(1, 2), (2, 3), (1, 4), (2, 4), (1, 3), (3, 4)]
And data, a 6-element Array{Float64,1}:
 11.0
  0.0
  0.0
  0.0
  0.0
 10.0


In [904]:
load_payment = sum(prices .* value.(d))
gen_revenue = sum(prices .* value.(p))
vl_revenue = value(sum(δ[(i,j)] * (prices[i] - prices[j]) for (i,j) in vls))
println("total load payment: $(load_payment)")
println("total generation revenue: $(gen_revenue)")
println("total VL revenue: $(vl_revenue)")
println("total revenue: $(gen_revenue + vl_revenue)")

total load payment: 5810.0
total generation revenue: 5570.0
total VL revenue: 240.0
total revenue: 5810.0


In [894]:
load_profit = sum((α_d .- prices) .* value.(d))
# vl_profit = value(sum((Del[i,j] > 0) * δ[i,j] * (prices[i] + ω[i] - prices[j] - ω[j] - α_del[i,j]) for i in 1:N for j in 1:N))
vl_profit = value(sum(δ[(i,j)] * (prices[i] - prices[j] - α_del[(i,j)]) for (i,j) in vls))
g_profit = value(sum((prices .- α_p) .* p))
println("total load profit: $(load_profit)")
println("total vl profit: $(vl_profit)")
println("gen profit: $(g_profit)")

total load profit: 1920.0
total vl profit: 177.0
gen profit: 3100.0


# 3. Space-Time Case

In [75]:
using PowerModels
using Random

In [1040]:
# file = "/home/weiqizhang/anl/pglib-opf/pglib_opf_case30_ieee.m";
file = "/home/weiqizhang/anl/pglib-opf/api/pglib_opf_case30_ieee__api.m";
# file = "/home/weiqizhang/anl/pglib-opf/api/pglib_opf_case24_ieee_rts__api.m";
data = parse_file(file);

[info | PowerModels]: removing 3 cost terms from generator 4: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 1: [1842.1527999999998, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 5: Float64[]
[info | PowerModels]: removing 1 cost terms from generator 2: [5218.2254, 0.0]
[info | PowerModels]: removing 3 cost terms from generator 6: Float64[]
[info | PowerModels]: removing 3 cost terms from generator 3: Float64[]


In [1041]:
# construct own data
buses = Dict{Int64,Dict}(v["bus_i"] => Dict{String, Set}("edges_in" => Set(Int[]),
                                                         "edges_out" => Set(Int[]),
                                                         "loads" => Set(Int[]),
                                                         "gens" => Set(Int[])) for (_,v) in data["bus"])
branches = Dict{Int64, Dict}()
for (_,v) in data["branch"]
    d = Dict{String, Any}("r" => v["br_r"], "x" => v["br_x"], "cap" => v["rate_a"], "f_bus" => v["f_bus"], "t_bus" => v["t_bus"], "angmin" => v["angmin"], "angmax" => v["angmax"])
    d["b"] = -imag(1 / (d["r"] + im * d["x"]))
    push!(buses[d["f_bus"]]["edges_out"], v["index"])
    push!(buses[d["t_bus"]]["edges_in"], v["index"])
    branches[v["index"]] = d
end
gens = Dict{Int64, Dict}()
for (_,v) in data["gen"]
#     if v["ncost"] > 0
        d = Dict{String, Any}("cap" => v["pmax"], "bus" => v["gen_bus"], "ncost" => v["ncost"], "cost" => v["cost"])
        push!(buses[d["bus"]]["gens"], v["index"])
        gens[v["index"]] = d
#     end
end
loads = Dict{Int64, Dict}()
for (_,v) in data["load"]
    d = Dict{String, Any}("bus" => v["load_bus"], "cap" => v["pd"])
    push!(buses[d["bus"]]["loads"], v["index"])
    loads[v["index"]] = d
end

In [1042]:
# add time-dependent data
# assume only load bid is dependent on time for now
T = 24
Random.seed!(0);
for (_,v) in loads
    set_point = v["cap"]
#     perc = rand(T) .+ 0.5
    perc = 0.5 * rand(T) .+ 0.75
    v["cap"] = set_point .* perc
end


# Assign bid information
for (k,v) in branches
    v["bid_cost"] = 0.1
end
for (k,v) in gens
    if v["ncost"] == 0
        v["bid_cost"] = 0
    else
        v["bid_cost"] = v["cost"][end-1]
    end
end
for (k,v) in loads
    v["bid_cost"] = repeat([20000.], T)
end

In [1043]:
# add space-time virtual links
dcnodes = [4,14,21,23,26,30]
vls = Dict{Int64, Dict}()
# vcap = 0.01
vcap = 0.0
dcap = 0.5

ct = 1
for i in 1:length(dcnodes)
    ni = dcnodes[i]
    for j in (i+1):length(dcnodes)
        nj = dcnodes[j]
        for t1 in 1:T
            for t2 in t1:T
                vls[ct] = Dict("source_bus" => ni, "source_time" => t1, "end_bus" => nj, "end_time" => t2, "bid_cost" => 0, "cap" => vcap)
                ct += 1
                vls[ct] = Dict("source_bus" => nj, "source_time" => t1, "end_bus" => ni, "end_time" => t2, "bid_cost" => 0, "cap" => vcap)
                ct += 1
            end
        end
    end
    for t1 in 1:T
        for t2 in (t1+1):T
            vls[ct] = Dict("source_bus" => ni, "source_time" => t1, "end_bus" => ni, "end_time" => t2, "bid_cost" => 0, "cap" => vcap)
            ct += 1
        end
    end

end


# vls[1] = Dict("source_bus" => 1, "source_time" => 1, "end_bus" => 2, "end_time" => 2, "bid_cost" => 10, "cap" => 0)

st_nodes = Dict{Tuple{Int64, Int64}, Dict}((i,t) => Dict{String, Set}("vls_in" => Set(Int[]), 
                                                                  "vls_out" => Set(Int[]))
                                           for i in keys(buses) for t in 1:T)
for (i,v) in vls
    push!(st_nodes[(v["end_bus"], v["end_time"])]["vls_in"], i)
    push!(st_nodes[(v["source_bus"], v["source_time"])]["vls_out"], i)
end

In [1044]:
# space-time market formulation

nbus = length(buses);
nbranch = length(branches);
ngen = length(gens);
nload = length(loads);
nvls = length(vls)
m = Model(Gurobi.Optimizer)
@variable(m, s[1:ngen, 1:T] >= 0)                      # supply
@variable(m, d[1:nload, 1:T] >= 0)                     # demand
@variable(m, ff[1:nbranch, 1:T] >= 0)                  # flow forward
@variable(m, fb[1:nbranch, 1:T] >= 0)                  # flow backward
@variable(m, θ[1:nbus, 1:T])
@variable(m, del[1:nvls] >= 0)                         # virtual shift

bals = Dict()
for t in 1:T
    # Supply and demand capacity constraints
    for (k,v) in gens
        @constraint(m, s[k,t] <= v["cap"])
    end
    for (k,v) in loads
        @constraint(m, d[k,t] <= v["cap"][t])
    end

    # Flow capacity constraints & DC power flow equations
    for (k,v) in branches
        @constraint(m, ff[k,t] <= v["cap"])
        @constraint(m, fb[k,t] <= v["cap"])
        @constraint(m, ff[k,t] - fb[k,t] == v["b"] * (θ[v["f_bus"],t] - θ[v["t_bus"],t]))
    end

    # Node balance constraint
    for (k,v) in buses
        bals[(k,t)] = @constraint(m, sum(s[i,t] for i in buses[k]["gens"]) - sum(d[i,t] for i in buses[k]["loads"])
                               + sum(ff[i,t] for i in buses[k]["edges_in"]) - sum(ff[i,t] for i in buses[k]["edges_out"])
                               - sum(fb[i,t] for i in buses[k]["edges_in"]) + sum(fb[i,t] for i in buses[k]["edges_out"]) 
                               - sum(del[i] for i in st_nodes[(k,t)]["vls_in"]) + sum(del[i] for i in st_nodes[(k,t)]["vls_out"])
                               == 0)
    end
end

# Virtual link capacity constraints
for i in 1:nvls
    @constraint(m, del[i] <= vls[i]["cap"])
end

# Computing capacity constraints
for k in dcnodes, t in 1:T
    @constraint(m, sum(d[i,t] for i in buses[k]["loads"]) + sum(del[i] for i in st_nodes[(k,t)]["vls_in"]) - sum(del[i] for i in st_nodes[(k,t)]["vls_out"]) >= 0)
    @constraint(m, sum(d[i,t] for i in buses[k]["loads"]) + sum(del[i] for i in st_nodes[(k,t)]["vls_in"]) - sum(del[i] for i in st_nodes[(k,t)]["vls_out"]) <= dcap)
end

# Objecitve: social welfare
@objective(m, Min, sum(sum(s[i,t] * gens[i]["bid_cost"] for i in 1:ngen) 
                       - sum(d[i,t] * loads[i]["bid_cost"][t] for i in 1:nload) 
                       + sum((ff[i,t] + fb[i,t]) * branches[i]["bid_cost"] for i in 1:nbranch)
                       for t in 1:T) + sum(del[i] * vls[i]["bid_cost"] for i in 1:nvls));

Academic license - for non-commercial use only - expires 2021-06-10


In [1045]:
optimize!(m)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15264 rows, 13992 columns and 86016 nonzeros
Model fingerprint: 0x3f204ee1
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e-01, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 4e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 14304 rows and 11592 columns
Presolve time: 0.02s
Presolved: 960 rows, 2400 columns, 6984 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.096e+03
 Factor NZ  : 9.120e+03 (roughly 1 MByte of memory)
 Factor Ops : 1.011e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.22993766e+07 -3.20079349e+07  1.39e+00 8.54e+03  3.00e+04     0s
   1  -1.41511091e+

In [922]:
prices = zeros(nbus, T)
for i in 1:nbus, t in 1:T
    prices[i,t] = dual(bals[(i,t)])
end
prices_df = DataFrame(prices, [Symbol("T$(i)") for i in 1:T])
display(prices_df)
# display("text/plain", prices)
# display("text/csv", round.(prices, digits = 2))

,T1,T2,T3,T4,T5,T6,T7,T8,T9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1842.15,1842.15,1842.15,1842.15,1842.15,1842.15,1842.15,1842.15,1842.15
2,5218.23,5218.23,5218.23,5218.23,5218.23,5218.23,5218.23,5218.23,5218.23
3,3762.2,3762.2,3762.2,3762.2,3762.2,3872.64,3762.2,3762.2,3762.2
4,4221.8,4221.81,4221.81,4221.81,4221.81,4358.68,4221.81,4221.81,4221.81
5,4852.16,4852.16,4852.16,4852.16,4852.16,5567.12,4852.15,4852.15,4852.15
6,4458.88,4458.87,4458.87,4458.87,4458.87,4651.84,4458.87,4458.87,4458.87
7,4617.47,4617.46,4617.46,4617.46,4617.46,5020.9,4617.46,4617.46,4617.46
8,4458.46,4458.45,4458.45,4458.45,4458.45,4651.3,4458.45,4458.45,4458.45
9,4421.1,4421.07,4421.07,4421.07,4421.07,4605.1,4421.06,4421.06,4421.06


In [923]:
# space-time price statistics
mean_price = round(Statistics.mean(prices), digits = 2)
median_price = round(Statistics.median(prices), digits = 2)
max_price = round(maximum(prices), digits = 2)
min_price = round(minimum(prices), digits = 2)
price_std = round(Statistics.std(prices), digits = 2)
price_abs_dev = round(Statistics.mean(abs.(prices .- mean_price)))

println("Space-time average price: $(mean_price)")
println("Space-time median price: $(median_price)")
println("Space-time maximum price: $(max_price)")
println("Space-time minimum price: $(min_price)")
println("Space-time price std: $(price_std)")
println("Space-time price average deviation: $(price_abs_dev)")

Space-time average price: 4377.19
Space-time median price: 4422.45
Space-time maximum price: 5594.81
Space-time minimum price: 1842.15
Space-time price std: 528.7
Space-time price average deviation: 232.0


In [446]:
# spatial price statistics
mean_price = round.(Statistics.mean(prices, dims = 2)[:,1], digits = 2)
median_price = round.(Statistics.median(prices, dims = 2)[:,1], digits = 2)
max_price = round.(maximum(prices, dims = 2)[:,1], digits = 2)
min_price = round.(minimum(prices, dims = 2)[:,1], digits = 2)
range = max_price .- min_price
price_std = round.(Statistics.std(prices, dims = 2)[:,1], digits = 2)
price_abs_dev = round.(Statistics.mean(abs.(prices .- mean_price), dims = 2)[:,1], digits = 2)

df = DataFrame(mean = mean_price,
               median = median_price,
               max = max_price,
               min = min_price,
               range = range,
               std = price_std,
               abs_dev = price_abs_dev
              )
print(df)

30×7 DataFrame
│ Row │ mean    │ median  │ max     │ min     │ range   │ std     │ abs_dev │
│     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 1842.15 │ 1842.15 │ 1842.15 │ 1842.15 │ 0.0     │ 0.0     │ 0.0     │
│ 2   │ 5218.23 │ 5218.23 │ 5218.23 │ 5218.23 │ 0.0     │ 0.0     │ 0.0     │
│ 3   │ 3783.11 │ 3762.2  │ 3876.92 │ 3754.31 │ 122.61  │ 43.21   │ 32.83   │
│ 4   │ 4247.72 │ 4221.81 │ 4363.98 │ 4212.03 │ 151.95  │ 53.55   │ 40.69   │
│ 5   │ 4992.88 │ 4852.24 │ 5594.81 │ 4852.15 │ 742.66  │ 276.63  │ 209.93  │
│ 6   │ 4497.9  │ 4459.04 │ 4659.32 │ 4458.87 │ 200.45  │ 74.18   │ 56.15   │
│ 7   │ 4697.49 │ 4617.6  │ 5036.53 │ 4617.46 │ 419.07  │ 155.79  │ 118.15  │
│ 8   │ 4497.6  │ 4458.65 │ 4658.77 │ 4458.45 │ 200.32  │ 74.07   │ 56.04   │
│ 9   │ 4463.61 │ 4421.99 │ 4612.24 │ 4421.06 │ 191.18  │ 69.96   │ 53.94   │
│ 10  │ 4445.52 │ 4402.44 │ 4587.39 │ 4401.11 │ 1

In [207]:
# temporal price statistics
mean_price = round.(Statistics.mean(prices, dims = 1)[1,:], digits = 2)
median_price = round.(Statistics.median(prices, dims = 1)[1,:], digits = 2)
max_price = round.(maximum(prices, dims = 1)[1,:], digits = 2)
min_price = round.(minimum(prices, dims = 1)[1,:], digits = 2)
range = max_price .- min_price
price_std = round.(Statistics.std(prices, dims = 1)[1,:], digits = 2)
price_abs_dev = round.(Statistics.mean(abs.(prices' .- mean_price), dims = 2)[:,1], digits = 2)

df = DataFrame(mean = mean_price,
               median = median_price,
               max = max_price,
               min = min_price,
               range = range,
               std = price_std,
               abs_dev = price_abs_dev
              )
print(df)

24×7 DataFrame
│ Row │ mean    │ median  │ max     │ min     │ range   │ std     │ abs_dev │
│     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 6223.24 │ 4803.52 │ 20000.0 │ 1842.15 │ 18157.8 │ 4119.51 │ 2923.67 │
│ 2   │ 10521.0 │ 10478.7 │ 20000.0 │ 1842.15 │ 18157.8 │ 4377.33 │ 3133.52 │
│ 3   │ 7225.74 │ 6826.53 │ 20000.0 │ -584.65 │ 20584.7 │ 4566.55 │ 3235.56 │
│ 4   │ 4331.01 │ 4399.09 │ 5218.23 │ 1842.15 │ 3376.08 │ 518.89  │ 214.56  │
│ 5   │ 7225.74 │ 6826.54 │ 20000.0 │ -584.66 │ 20584.7 │ 4566.55 │ 3235.56 │
│ 6   │ 6223.24 │ 4803.52 │ 20000.0 │ 1842.15 │ 18157.8 │ 4119.51 │ 2923.67 │
│ 7   │ 7225.76 │ 6826.54 │ 20000.0 │ -584.57 │ 20584.6 │ 4566.53 │ 3235.56 │
│ 8   │ 8298.55 │ 8158.65 │ 20000.0 │ 1842.15 │ 18157.8 │ 2755.86 │ 1173.11 │
│ 9   │ 10521.0 │ 10478.7 │ 20000.0 │ 1842.15 │ 18157.8 │ 4377.33 │ 3133.52 │
│ 10  │ 10521.0 │ 10478.6 │ 20000.0 │ 1842.15 │ 1